In [4]:
import tensorflow as tf
from tensorflow.keras import models,layers
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

batch_size=32
image_size=256
channels = 3 
epochs =25 
dataset= tf.keras.utils.image_dataset_from_directory(directory="../PlantVillage",batch_size=batch_size, image_size=(image_size,image_size))

class_names=dataset.class_names

def  dataset_split(ds,train_split=0.8,val_split=0.1,test_split=0.1,shuffle_size=10000):
    ds_size=len(ds)
    
    if shuffle_size:
        ds=ds.shuffle(shuffle_size,seed=42)
        
    train_size=int(train_split*ds_size)
    val_size=int(val_split*ds_size)
    
    train_ds=ds.take(train_size)
    validation_ds=ds.skip(train_size).take(val_size)
    test_ds=ds.skip(train_size).skip(val_size)
    
    return train_ds ,validation_ds ,test_ds

train_ds , validation_ds ,test_ds=dataset_split(dataset)

plt.figure(figsize=(10,10))
for image_batch,label_batch in dataset.take(1):
    for i in range(12):
        ax=plt.subplot(3,4,i+1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.axis("off")
        plt.title(class_names[label_batch[i]])

train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=validation_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(image_size, image_size),
  layers.experimental.preprocessing.Rescaling(1./255)
])
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])

input_size=(batch_size,image_size,image_size,channels)
model = tf.keras.models.Sequential([
    data_augmentation,
    layers.Conv2D(16,(3,3),activation="relu",input_shape=input_size),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(32,(3,3),activation="relu"),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(32,(3,3),activation="relu"),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation="relu"),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation="relu"),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128,(3,3),activation="relu"),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64,activation="relu"),
    layers.Dense(3,activation="softmax")
])
model.build(input_shape=input_size)

model.summary()

model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=["accuracy"])

history=model.fit(train_ds,epochs=epochs,batch_size=batch_size,verbose=1,validation_data=val_ds)

model.evaluate(test_ds)

#Matplotlib
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
val_loss=history.history['val_loss']
loss=history.history['loss']

plt.figure(figsize=(8,4))
plt.plot(range(epochs),acc,label="Training Acc")
plt.plot(range(epochs),val_acc,label="Val Acc")
plt.legend(loc="lower right")
plt.title('Training and Validation Accuracy')
#============================================#
plt.figure(figsize=(8,4))
plt.plot(range(epochs),loss,label="Training loss")
plt.plot(range(epochs),val_loss,label="Val loss")
plt.legend(loc="lower left")
plt.title('Training and Validation loss')

#Prediction
for images_batch,label_batch in test_ds.take(1):
    
    first_image= images_batch[0].numpy().astype('uint8')
    first_label= label_batch[0].numpy()
    
    print("Firrst image to predict")
    plt.imshow(first_image)
    print("first image's actual label : ",class_names[first_label])
    
    batch_predict=model.predict(images_batch)
    print("first image's predicted label : ",class_names[np.argmax(batch_predict[0])])

#function to predict 
def predict_name (model,img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array,0) #create a batch
    
    predictions = model.predict(img_array)
    
    predicted_class = class_names[np.argmax(predictions[0])]
    cofidence = round(100*(np.max(predictions[0])),2)
    return predicted_class , cofidence

plt.figure(figsize=(15,15))
for images, labels in test_ds.take(1):  
    for i in range(9):
        ax=plt.subplot(3,4,i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.axis("off")
        
        predicted_class,cofidence = predict_name(model,images[i].numpy())
        actual_class = class_names[labels[i]]
        
        plt.title(f"Actual: {actual_class},\n Predicted : {predicted_class},\n Cofidence {cofidence}")

import os

os.getcwd()

model_version=1
model.save(f"../Saved_Models/{model_version}")





